# Download web images on a GCS Bucket

## 1. Instanciate Google Cloud Storage client

In [1]:
from google.cloud import storage

def upload_blob(bucket_name, source_file_name, destination_blob_name,imBytes ):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_file(imBytes)

def upload_blob_exists(fileName, blobsName ):
    if fileName in blobsName:
        return True

In [ ]:
upload_blob("BUCKET_NAME", "IMG_PATH","DESTINATION_PATH" )

In [2]:
from google.cloud import storage
import os
def list_blobs(bucket_name, prefix):
    """Lists all the blobs in the bucket."""
    # bucket_name = "your-bucket-name"

    storage_client = storage.Client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix)

    # Note: The call returns a response only when the iterator is consumed.
    blobs=[os.path.split(blob.name )[1] for blob in blobs]
    return blobs
        

## 2. Create a list of image urls and load each image in GCS

In [3]:
# The following two lines are only necessary to run once.
# Comment out otherwise for speed-up.
from google.cloud.bigquery import Client, QueryJobConfig
client = Client()

project_id= 'pod-fr-retail'
dataset='k_temp'
tablename='shopping'

query="""
select a uri
FROM `{}.{}.{}`, UNNEST(additional_image_link) a
where a is not null and a <> ""
GROUP BY 1
""".format(project_id ,dataset ,tablename )
job = client.query(query)
df = job.to_dataframe()

In [5]:
import os
import PIL
from PIL import Image
import glob
import requests
from io import BytesIO
import time
blobs=list_blobs("pod-fr-retail-kaggle", "kiabi/images/" )
print(len(blobs))
existing=0
new=0
maxBeforeWait=100
i=0
for url in df["uri"].values.tolist():
    filepath, filenameExt = os.path.split(url)
    upload_results_exists=upload_blob_exists(filenameExt, blobs)
    if upload_results_exists:
        existing=existing+1
        i=i+1
    else:
        response = requests.get(url)
        try:
            im = BytesIO(response.content)
        except PIL.UnidentifiedImageError:
            print(response.content)
        upload_blob("pod-fr-retail-kaggle", "images/"+filenameExt,"kiabi/images/"+filenameExt , im)
        new=new+1
        i=i+1
    if i>=maxBeforeWait:
        i=0
        #time.sleep(2)
        print(f"existing files:{existing}, new files:{new}")
print(f"existing files:{existing}, new files:{new}")

90635
existing files:100, new files:0
existing files:200, new files:0
existing files:300, new files:0
existing files:400, new files:0
existing files:500, new files:0
existing files:600, new files:0
existing files:700, new files:0
existing files:800, new files:0
existing files:900, new files:0
existing files:1000, new files:0
existing files:1100, new files:0
existing files:1200, new files:0
existing files:1300, new files:0
existing files:1400, new files:0
existing files:1500, new files:0
existing files:1600, new files:0
existing files:1700, new files:0
existing files:1800, new files:0
existing files:1900, new files:0
existing files:2000, new files:0
existing files:2100, new files:0
existing files:2200, new files:0
existing files:2300, new files:0
existing files:2400, new files:0
existing files:2500, new files:0
existing files:2600, new files:0
existing files:2700, new files:0
existing files:2800, new files:0
existing files:2900, new files:0
existing files:3000, new files:0
existing file